In [1]:
import pickle as pck
from tqdm.notebook import tqdm
from collections import *
from dateutil.parser import parse
from multiprocessing import Pool
from multiprocessing.dummy import Pool as DPool
from copy import deepcopy
import matplotlib.pyplot as plt
import json
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font

import pandas as pd
import numpy as np
import swifter
import matplotlib as mpl
import matplotlib.backends.backend_pdf
import re
from numba import jit
import pymorphy2
import math
import gensim
from typing import List, Tuple
import openpyxl
from googletrans import Translator
import time
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
from datetime import *
from sklearn.metrics.pairwise import cosine_similarity
import plotly.io as pi

pio.orca.config.use_xvfb = True
pio.orca.config.use_xvfb = False
mpl.rc("savefig", dpi=200)
morph = pymorphy2.MorphAnalyzer()
translator = Translator()

pd.options.display.max_columns = 500
pd.options.display.max_rows = 300

import xlsxwriter

%matplotlib inline

In [2]:
tmp_vector = np.array([0] * 300, dtype=np.float32)
model = gensim.models.KeyedVectors.load_word2vec_format('../hh_data/model.bin', binary=True)
def _word2vec(word):
    for i in ["_NOUN", "_ADJ", "_VERB"]:
        tmp = "{}{}".format(word, i)
        if tmp in model:
            return model[tmp]

    return np.array(tmp_vector)

def normalize_word(word):
    norm_words = morph.parse(word)
    
    res = None
    
    for norm_word in norm_words:
        # Если слово не предлог и не союз
        # Не в стоп словах и длинна слова больше одного символа
        if word in ['не', 'нет', 'без'] or norm_word.tag.POS not in ['PREP', 'CONJ', 'PRCL', 'INTJ'] \
            and norm_word.normal_form and len(norm_word.normal_form) > 1 and norm_word.score > 0.1 \
            and (np.linalg.norm(_word2vec(norm_word.normal_form)) > 0 or norm_word.normal_form in neutral + positive + negative):

            # Сохраним первую форму слова
            res = norm_word.normal_form
            break
    return morph.parse(word.lower())[0].normal_form


negative = ['нестабильный', 'вышеднем', 'коронабесные', 'коронаканикулы', 'страшно', 'выгорание', 'тупо', 'демотивирован', 'закрываю', 'инфоцыган', 'сложно', 'кошмарят', 'банкротства', 'банкротству', 'некомпетентности', 'некомпетентностью', 'игнорируются', 'нельзя', 'нервы', 'обязаловки', 'убыль', 'Позорище', 'коронакризис', 'скормил', 'против', 'экономить', 'херь']
neutral = ['электрон', 'мире', 'Видя', 'рекламщиками', 'штурм', 'вижу', 'вышло', 'подчиненные', 'долгожданное', 'дороже', 'кролик', 'коммуналку', 'Медузы', 'опять', 'отложенный', 'отмены', 'спишут', 'нужно', 'очередь', 'мелкими', 'подвести', 'душу', 'подчинении', 'чуваком', 'политики', 'пошли', 'разворачивание', 'топлю', 'рекламы', 'экология', 'следить', 'стихия', 'природа', 'Цепями']
positive = ['успешно', 'идеально', 'добра', 'доделали', 'запуск', 'Запускаю', 'кайфе', 'легко', 'наконец', 'можно', 'открыть', 'продажи', 'прорвёмся', 'удобно', 'ТОП', 'Учреждена', 'возобновил', 'открыты']

negative = [normalize_word(word) for word in negative]
neutral = [normalize_word(word) for word in neutral]
positive = [normalize_word(word) for word in positive]

emo = pd.read_csv("emo_dict.csv", sep=';')


def genRow(term, kind):
    if kind == 'NEUT':
        values = {'tag':'NEUT', 
            'value':0.0, 
            'pstv':0.0000, 
            'neut':1.0, 
            'ngtv':0.0}
        
    elif kind == 'PSTV':
        values = {'tag':'PSTV', 
            'value':1.0, 
            'pstv':1.0000, 
            'neut':0.0, 
            'ngtv':0.0}
    else:
        values = {'tag':'NGTV', 
            'value':-1.0, 
            'pstv':0.0000, 
            'neut':0.0, 
            'ngtv':-1.0}
        
    return {'term':term, 
            **values, 
            'dunno':0.000,
            'distortion':0.0000}

def add_word(df, word, kind):
    df = df.drop(df[df.term == word].index)
    df = df.append(genRow(word, kind), ignore_index = True)
    return df

for neg in negative:
    emo = add_word(emo, neg, 'NGTV')
    assert(len(emo[emo.term == neg]) == 1 and emo[emo.term == neg].value.values[0] == -1)
    
for neut in neutral:
    emo = add_word(emo, neut, 'NEUT')
    assert(len(emo[emo.term == neut]) == 1 and emo[emo.term == neut].value.values[0] == 0)

for pstv in positive:
    emo = add_word(emo, pstv, 'PSTV')
    assert(len(emo[emo.term == pstv]) == 1 and emo[emo.term == pstv].value.values[0] == 1)
        
emo.head()

,term,tag,value,pstv,neut,ngtv,dunno,distortion
0,аббат,NEUT,0.3667,0.2574,0.3762,0.0693,0.2970,0.0880
1,аббревиатура,NEUT,0.0000,0.1600,0.7200,0.0000,0.1200,0.0000
2,абзац,NEUT,0.0000,0.1481,0.7037,0.0000,0.1481,0.0000
3,абонемент,NEUT,0.1757,0.2381,0.5810,0.0476,0.1333,0.0571
4,абонентный,NEUT,0.0000,0.0000,0.7200,0.0000,0.2800,0.0000


In [3]:
value = float(emo[emo.term == 'зарплата'].value.values[0])
emo.loc[emo.term == 'зарплата', 'value'] = -0.9
emo.loc[emo.term == 'зарплата', 'tag'] = 'NGTV'
emo.loc[emo.term == 'зарплата', 'pstv'] = 0.0294
emo.loc[emo.term == 'зарплата', 'ngtv'] = 0.7059

value = float(emo[emo.term == 'каникулы'].value.values[0])
emo.loc[emo.term == 'каникулы', 'value'] = 0.651

emo[(emo.term == 'зарплата') | (emo.term == 'каникулы')]

,term,tag,value,pstv,neut,ngtv,dunno,distortion
6602,зарплата,NGTV,-0.900,0.0294,0.2353,0.7059,0.0294,0.0306
7914,каникулы,PSTV,0.651,0.8621,0.1034,0.0345,0.0000,0.0359


In [4]:
words_norm_dict = {}

def clean_text(name):
    '''Первичная очистка названия вакансии'''
    
    # Приведем к нижнему регистру
    name = name.lower().replace('...', ' ').replace("ё", 'е')
    
    # Оставим только буквы
    name = re.sub(r"[^a-zа-я]+", ' ', name).strip()
    
    return name

def normalize_text(name):
    
    normal_name = ""
    name = name.replace("\n", " ")
    # Для каждого слова в вакансии
    for word in name.split():
        
        found_norm_word = None
        
        # Если слово не в кэше
        if word not in words_norm_dict.keys():
            # Распарсим его
            found_norm_word = normalize_word(word)
                    
            # Запишем слово в кэш
            words_norm_dict[word] = found_norm_word
        else:
            # Если слово находится в кэше - просто достанем результат
            found_norm_word = words_norm_dict[word]

        # Если результат не отрицательный и слово есть
        if found_norm_word:
            # Запишим его в нормализованную вакнасию
            normal_name = f"{normal_name} {found_norm_word}"
            
    normal_name = normal_name.strip()
    return normal_name

In [10]:
data1 = pd.read_csv("posts_cv3.csv").fillna("") 
data = pd.concat([data1])
print(len(data))
data.head()

400


,Ссылка на источник,Ссылка на пост,Текст поста,Коэффициент поста,Тема 1,Тема 2,Тема 3
0,https://www.facebook.com/bablorub,https://www.facebook.com/bablorub/posts/368965...,Как выживает ресторанная индустрия.\nДа никак....,,Бизнес справился с кризисом,Нехватка средств / закрытие бизнеса,
1,https://www.facebook.com/bablorub,https://www.facebook.com/bablorub/posts/361902...,"Ивентер 2019: ты едешь на велосипеде, который ...",,"Неопределенность, страх, выгорание из-за пандемии",,
2,https://www.facebook.com/bablorub,https://www.facebook.com/bablorub/posts/360052...,Отложенный спрос работал только пару недель по...,,Снижение потребительского спроса,,
3,https://www.facebook.com/bablorub,https://www.facebook.com/bablorub/posts/359459...,"На глазах рождается новый анекдот.\n- Деда, а ...",,Нехватка средств / закрытие бизнеса,,
4,https://www.facebook.com/bablorub,https://www.facebook.com/bablorub/posts/357796...,Что произошло с Васей и Петей и их бизнесами? ...,,Снижение потребительского спроса,Необходимость адаптироваться под новые условия,


In [11]:
texts = data['Текст поста']

In [12]:
norm_texts = [normalize_text(clean_text(text)) for text in tqdm(texts)]

In [13]:
words = []
for text in norm_texts:
    words.extend(text.split())
words = list(set(words))

In [14]:
len(words)

9530

In [15]:
word_vectors = []
for word in words:
    word_vectors.append(_word2vec(word))
word_vectors = np.array(word_vectors)


In [16]:
words[0], len(word_vectors[0])

('разницой', 300)

In [17]:
emo_terms, emo_values = emo.term.values, emo.value.values

In [18]:
emo_dict = {}
emo_vectors = []
for i in range(len(emo_terms)):
    emo_dict[emo_terms[i]] = emo_values[i]
    emo_vectors.append(_word2vec(emo_terms[i]))
emo_vectors = np.array(emo_vectors)

In [19]:
word_vectors.shape, emo_vectors.shape

((9530, 300), (28223, 300))

In [20]:
# матрица косинусных расстояний
cos = cosine_similarity(word_vectors, emo_vectors)

In [21]:
cos.shape

(9530, 28223)

In [22]:
for index in range(len(words)):
    if words[index] in emo_terms:
        print(index, np.where(emo_terms==words[index])[0][0])
        break

1 27853


In [23]:
#отношение слово из постов - слово из словаря
word_connection = []
for word_ind in range(len(words)):
    word_connection.append(np.argmax(cos[word_ind]))
    

In [24]:
print("слово из поста; слово из словаря; cos; тональность")
for i in range(len(word_connection[:10])):
    print(words[i], emo_terms[word_connection[i]], cos[i][word_connection[i]], emo_dict[emo_terms[word_connection[i]]], sep='; ')

слово из поста; слово из словаря; cos; тональность
разницой; аббат; 0.0; 0.3667
экологический; экологический; 0.99999976; 0.7925
ндфл; аббат; 0.0; 0.3667
medium; аббат; 0.0; 0.3667
традиция; традиция; 1.0; 0.6364
выкладка; рассуждение; 0.4888876; 0.5301
кредиторский; кредиторский; 1.0000001; 0.0
парковый; парковый; 1.0000001; 0.09300000000000001
просить; просить; 0.9999999; 0.0893
дата; дата; 0.99999976; 0.0


In [25]:
words_emos = dict()
for i in range(len(word_connection)):
    if words[i] in emo_dict:
        value = emo_dict[words[i]]
        if value <= 0.65 and value > 0:
            value = 0
        elif value < 0:
            value -= (value+1)/2
        words_emos[words[i]] = value
        continue
        
    if cos[i][word_connection[i]] < 0.7:
        words_emos[words[i]] = 0
    else:
        value = emo_dict[emo_terms[word_connection[i]]]
        if value <= 0.651 and value > 0:
            value = 0
        elif value < 0:
            value -= (value+1)/2
        words_emos[words[i]] = value

In [26]:
cnt = 0
for word in words_emos:
    if words_emos[word] != 0:
        cnt += 1
f"{round(cnt / len(words_emos) * 100, 2)}%"

'20.89%'

In [27]:
words_emos[words_norm_dict['позорище']]

-1.0

In [28]:
norm_texts_values = []
for text in norm_texts:
    value = 0
    cnt = 0
    prevNot = 1
    for word in text.split():
        if words_emos[word] != 0:
            cnt+=1
            value += words_emos[word] * prevNot
            
        prevNot = 1
        
        if word == 'не':
            prevNot = -1
            
    norm_texts_values.append(value / cnt if cnt > 0 else 0)
norm_texts_values = np.array(norm_texts_values)

In [29]:
norm_texts_values.mean()

0.3628703701370506

In [30]:
def get_color(c):
    if c >= 0:
        return '#%02x%02x%02x' % (0, int(c * 255), 0)
    else:
        return '#%02x%02x%02x' % (int(-c * 255), 0, 0)

In [31]:
color_texts = []
for text in tqdm(texts):
    fixed_text = text.replace('\n', '\n ').replace(".", '. ')
    col_t = []
    prevNot = 1
    for i, word in enumerate(fixed_text.split(" ")):
        lower_word = clean_text(word).split(' ')[0]
        if len(word) == 0:
            continue
        
        if lower_word in words_norm_dict and words_norm_dict[lower_word]:
            if prevNot == -1:
                col_t[-2] = words_emos[words_norm_dict[lower_word]] * prevNot
                    
            col_t.append(words_emos[words_norm_dict[lower_word]] * prevNot)
        else:
            col_t.append(0)
        prevNot = 1
        
        if word.lower() in ['не', 'нет', 'без']:
            prevNot = -1
            
        col_t.append(f"{word} ")
    color_texts.append(col_t)

In [32]:
words_emos[words_norm_dict['каникулы']]

0.651

In [33]:
name = '/mnt/Modulebank/ans.xlsx'
workbook = xlsxwriter.Workbook(name)
worksheet = workbook.add_worksheet()

In [34]:
scr_links = data['Ссылка на источник'].values
post_links = data['Ссылка на пост'].values

worksheet.write(0, 0, 'Ссылка на источник')
worksheet.write(0, 1, 'Ссылка на пост')
worksheet.write(0, 2, 'Текст поста')
worksheet.write(0, 3, 'Коэффициент поста')

for i, text in tqdm(enumerate(color_texts), total=len(color_texts)):
    
    index = i+1
    
    worksheet.write(index, 0, scr_links[i])
    worksheet.write(index, 1, post_links[i])
        
    if len(text):
        text_fmt = [workbook.add_format({'color': get_color(i)}) if type(i) in [int, np.float64] else i for i in text]
        res = worksheet.write_rich_string(f'C{index+1}',*text_fmt)
        if res != 0:
            print(i, res)
    
    worksheet.write(index, 3, norm_texts_values[i])
    
worksheet.write(1, 4, 'Итоговый коэффициент')          
worksheet.write_formula(f'F2', f'=AVERAGE(D1:D{len(texts)})')
workbook.close()

In [35]:
index = words.index('свой')
print(index, cos[index].argmax(), cos[index][cos[index].argmax()])


9137 22503 1.0000001


In [36]:
emo_terms[22541], emo_values[22541]

('сглаживать', 0.732)

In [37]:
words_emos['добро']

1.0

In [38]:
morph.parse('электрон')[0].normal_form

'электрон'

In [39]:
def get_info(word):
    word = morph.parse(word)[0].normal_form
    print("Слово", word)
    word_index = words.index(word)
    emo_index = word_connection[word_index]
    print("Ближайшее", emo_terms[emo_index], emo_dict[emo_terms[emo_index]])
    print("РАсстояние:", cos[word_index][emo_index])

In [40]:
get_info('добро')

Слово добро
Ближайшее добро 1.0
РАсстояние: 1.0000002


In [41]:
'беременность' in emo_terms

True

In [42]:
emo[emo.term == 'беременность']

,term,tag,value,pstv,neut,ngtv,dunno,distortion
909,беременность,PSTV,0.657,0.5146,0.3301,0.1262,0.0291,0.1572


## Частотность

In [46]:
words_cnt = Counter()
for text in norm_texts:
    for word in text.split():
        if words_emos[word]:
            words_cnt[word] += 1
to_frec = []
for word in words_cnt.most_common():
    to_frec.append([word[0], word[1], words_emos[word[0]], 'положительное' if words_emos[word[0]] > 0 else 'отрицательное'])


In [47]:
freq_data = pd.DataFrame(to_frec, columns=['Слово', 'Кол-во употреблений', 'Коэффициент', 'Тональность'])
freq_data.to_csv('words_freq.csv', decimal=',')
freq_data.head()

,Слово,Кол-во употреблений,Коэффициент,Тональность
0,налог,148,-0.71295,отрицательное
1,хороший,144,1.00000,положительное
2,можно,132,1.00000,положительное
3,жизнь,93,0.80920,положительное
4,знать,81,0.69050,положительное


In [45]:
sum([i[1] for i in words_cnt.most_common()])

8355